In [9]:

import numpy as np
import pandas as pd
from pyntcloud import PyntCloud

In [103]:
%%time
import numpy as np
np.set_printoptions(precision=3)
def simulate(aa, bb, N=2000):
    w = np.array([0, 1, 0], dtype = np.int32)
    s = np.array([0, -1, 0], dtype = np.int32)
    d = np.array([1, 0, 0], dtype = np.int32)
    a = np.array([-1, 0, 0], dtype = np.int32)
    e = np.array([0, 0, 1], dtype = np.int32)
    q = np.array([0, 0, -1], dtype = np.int32)

    class NDSparseLattice:
        def __init__(self):
            self.elements = {}

        def addValue(self, tuple, value):
            self.elements[tuple] = value

        def readValue(self, tuple):
            try:
                value = self.elements[tuple]
            except KeyError:
                # could also be 0.0 if using floats...
                value = 1
            return value

    def get_near(lattice, pos):
        n_w = lattice.readValue((pos[0], pos[1]+1, pos[2]))
        n_s = lattice.readValue((pos[0], pos[1]-1, pos[2]))
        n_d = lattice.readValue((pos[0]+1, pos[1], pos[2]))
        n_a = lattice.readValue((pos[0]-1, pos[1], pos[2]))
        n_e = lattice.readValue((pos[0], pos[1], pos[2]+1))
        n_q = lattice.readValue((pos[0], pos[1], pos[2]-1))
        return [n_w, n_s, n_d, n_a, n_e, n_q]

    def number_of_zero_neib(lattice, pos):
        if lattice.readValue((pos[0], pos[1], pos[2])) == 0:
            return 0
        else:
            count = 0
            array = get_near(lattice, pos)
            for item in array:
                if item == 0:
                    count = count+1
            return count+1

    def update_lattice(lattice, pos):
        lattice.addValue((pos[0], pos[1], pos[2]), 0)
        lattice.addValue((pos[0], pos[1]+1, pos[2]), number_of_zero_neib(lattice, pos+w))
        lattice.addValue((pos[0], pos[1]-1, pos[2]), number_of_zero_neib(lattice, pos+s))
        lattice.addValue((pos[0]+1, pos[1], pos[2]), number_of_zero_neib(lattice, pos+d))
        lattice.addValue((pos[0]-1, pos[1], pos[2]), number_of_zero_neib(lattice, pos+a))
        lattice.addValue((pos[0], pos[1], pos[2]+1), number_of_zero_neib(lattice, pos+e))
        lattice.addValue((pos[0], pos[1], pos[2]-1), number_of_zero_neib(lattice, pos+q))
        return lattice


    def weight(x, aa, bb):
        if x==0:
            return aa
        elif x==2:
            return 1
        elif x==1:
            print('Error')
            raise TypeError
        else:
            return bb

    def get_prob(lattice, pos, aa, bb):
        near = get_near(lattice, pos)
        norm = sum(near)
        probs = [weight(_, aa, bb) for _ in near]
        norm = sum(probs)
        probs_normed = [_/norm for _ in probs]
        return probs_normed

    def get_next_pos():
        symbol = np.random.choice(['w', 's', 'd', 'a', 'e', 'q'], p = get_prob(lattice, pos, aa, bb))
        if symbol == 'w':
            return w
        if symbol == 's':
            return s
        if symbol == 'd':
            return d
        if symbol == 'a':
            return a
        if symbol == 'e':
            return e
        if symbol == 'q':
            return q


    lattice = NDSparseLattice()
    pos = np.array([0,0,0], dtype=np.int32)
    lattice = update_lattice(lattice, pos)
    #visited
    
    from progressbar import ProgressBar
    pbar = ProgressBar()
    
    #for _ in pbar(range(N)):
    #    #visited.append(list(pos))
    #    pos = pos +  get_next_pos()
    #    lattice = update_lattice(lattice, pos)
    cnt = 0
    while len(lattice.elements)<N:
        pos = pos +  get_next_pos()
        lattice = update_lattice(lattice, pos)
        cnt += 1
    np.set_printoptions(precision=3)
    print(np.array([np.log(aa), np.log(bb)]), ':', cnt)
    return lattice, cnt, np.log(aa), np.log(bb)

def simulate_vector(vector, N=2000):
    return simulate(vector[0], vector[1], N=N)


lat, cnt, aa, bb = simulate_vector([np.exp(3), np.exp(-3)], N=1000000)

[ 3. -3.] : 19067475
Wall time: 40min 25s


In [104]:
coords = list(lat.elements.keys())
vals = list(lat.elements.values())
coords = [list(coords[_]) for _ in range(len(coords)) if (vals[_] != 1)]
import numpy as np
coords = np.array(coords)

In [105]:
coords

array([[   0,    0,    0],
       [   0,    1,    0],
       [   0,   -1,    0],
       ...,
       [ 142,  101, -259],
       [ 141,  102, -259],
       [ 142,  102, -258]])

In [106]:
np.max(np.abs(coords))

292

In [107]:
positions = np.random.rand(1000, 3) * 10
positions -= positions.mean(0)

points = pd.DataFrame(
    coords.astype(np.float32), 
    columns=['x', 'y', 'z'])

max_dist = np.max(np.abs(coords))

points["red"] = ((coords[:,0]*90)//max_dist - 255//2).astype(np.uint8)
points["green"] = ((coords[:,1]*90)//max_dist - 255//2).astype(np.uint8)
points["blue"] = ((coords[:,2]*90)//max_dist - 255//2).astype(np.uint8)

cloud = PyntCloud(points)

In [109]:
k_neighbors = cloud.get_neighbors(k=350)

In [110]:
eigenvalues = cloud.add_scalar_field(
    "eigen_values", 
    k_neighbors=k_neighbors)

In [111]:
curvature = cloud.add_scalar_field("curvature", ev=eigenvalues)

In [113]:
cloud.plot(use_as_color=curvature, cmap="plasma", backend='pythreejs')

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(47.650177001953125, 198.1258087158203, 12.90…

In [114]:
sphericity = cloud.add_scalar_field("sphericity", ev=eigenvalues)

In [115]:
cloud.plot(use_as_color=sphericity, cmap="plasma")

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(47.650177001953125, 198.1258087158203, 12.90…

In [108]:
cloud.plot(mesh=False, backend="pythreejs", elev=130.0, azim=120.0, cmap='plasma')

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(47.650177001953125, 198.1258087158203, 12.90…

In [102]:
cloud.plot(mesh=False, backend="pythreejs", elev=130.0, azim=120.0, cmap='plasma')

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-9.917013168334961, 419.87032318115234, -314…

In [96]:
cloud.plot(mesh=False, backend="pythreejs", elev=130.0, azim=120.0, cmap='plasma')

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-28.35145378112793, 36.00948715209961, 10.89…

In [89]:
cloud.plot(mesh=False, backend="pythreejs", elev=130.0, azim=120.0, cmap='plasma')

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-205.6302490234375, 425.5820617675781, -99.7…

In [82]:
cloud.plot(mesh=False, backend="pythreejs", elev=130.0, azim=120.0, cmap='plasma')

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-216.8092498779297, 297.9936828613281, -463.…

In [40]:
fig

In [18]:
import pandas as pd

points = pd.DataFrame(
    positions.astype(np.float32), 
    columns=['x', 'y', 'z'])

points["red"] = (np.random.rand(1000) * 255).astype(np.uint8)
points["green"] = (np.random.rand(1000) * 255).astype(np.uint8)
points["blue"] = (np.random.rand(1000) * 255).astype(np.uint8)

,x,y,z,red,green,blue
0,0.506345,0.774733,-2.201511,45,24,42
1,-4.346781,3.903501,-0.295702,27,9,82
2,-3.942817,-2.770571,-3.761680,78,240,237
3,0.770353,1.731843,1.111662,246,146,206
4,1.662170,1.487582,4.309238,67,103,49
...,...,...,...,...,...,...
995,-2.755202,-4.137967,0.252708,123,17,0
996,2.030230,1.355620,2.622153,47,211,146
997,-3.230269,-0.694178,0.355820,31,190,173
998,-4.950685,-3.728886,0.404361,227,109,204
